In [1]:
# Add PyDatAnalysis to path
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, "/Users/owensheekey/Documents/Research/PyDatAnalysis")

export_path = 'Exports/'

In [2]:
from __future__ import annotations
from src.DatObject.Make_Dat import get_dat, get_dats
import src.UsefulFunctions as U
from src.DataStandardize.ExpSpecific.Feb21 import Feb21Exp2HDF, Feb21ExpConfig
from src.DataStandardize.ExpConfig import ExpConfigGroupDatAttribute, ExpConfigBase
import multiprocessing as mp
import plotly.graph_objs as go
import numpy as np
import lmfit as lm
from typing import TYPE_CHECKING, Iterable, Optional

In [3]:
def theta_vs_fridge_temp_fig(thetas: Iterable[float], temps: Iterable[float], datnums: Iterable[int],
                             lower_fit_temp_limit: Optional[float] = None) -> go.Figure:
    fig = go.Figure()
    fig.add_trace(go.Scatter(mode='markers+lines+text', x=temps, y=thetas, text=datnums, name='Avg Thetas'))
    if lower_fit_temp_limit is not None:
        line = lm.models.LinearModel()
        fit_thetas, fit_temps = np.array([(theta, temp) for theta, temp in zip(thetas, temps) if temp > lower_fit_temp_limit]).T
        fit = line.fit(fit_thetas, x=fit_temps, nan_policy='omit')
        x_range = np.linspace(0, max(temps), int(max(temps))+1)
        fig.add_trace(go.Scatter(mode='lines', x=x_range, y=fit.eval(x=x_range),
                                 name=f'Fit to temps > {lower_fit_temp_limit}'))
    fig.update_layout(xaxis_title='Temp /mK', yaxis_title='Theta /mV',
                      title=f'Dats{min(datnums)}-{max(datnums)}: Transition vs Fridge temp')
    return fig

In [7]:
dats = get_dats(np.array([824, 836, 848, 860, 872, 889, 901, 913, 926, 938, 958, 970, 982])-2, exp2hdf=Feb21Exp2HDF)
thetas_ = [dat.Transition.avg_fit.best_values.theta for dat in dats]
temps_ = [300, 275, 250, 225, 200, 175, 150, 125, 100, 75, 50, 30, 10]
datnums_ = [dat.datnum for dat in dats]
fig = theta_vs_fridge_temp_fig(thetas=thetas_, temps=temps_, datnums=datnums_, lower_fit_temp_limit=160)
fig.show(renderer='browser')

In [9]:
def theta_vs_heaterbias_fig(thetas: Iterable[float], heaters: Iterable[float], datnums: Iterable[int], BD2D: float):
    fig = go.Figure()
    fig.add_trace(go.Scatter(mode='markers+lines+text', x=heaters, y=thetas, text=datnums, name='BD2D = {BD2D} mV'))
    fig.update_layout(xaxis_title='Heater bias /nA', yaxis_title='Theta /mV',
                      title=f'Dats{min(datnums)}-{max(datnums)}: DC bias, BD2D = {BD2D} mV')
    return fig

In [ ]:
dats = get_dats((1017, 1017+9*8), exp2hdf=Feb21Exp2HDF)
heaters_ = np.array([0, 5, 10, 20, 30, 40, 50, 75, 100])/10
fig = go.Figure()
for i in range(8):
    dats = get_dats((1017+9*i, 1017+i*9+9), exp2hdf=Feb21Exp2HDF)
    datnums_ = [dat.datnum for dat in dats]
    thetas_ = [dat.Transition.avg_fit.best_values.theta for dat in dats]
    fig.add_trace(go.Scatter(mode='markers+lines+text', x=heaters_, y=thetas_, text=datnums_, name='BD2D = {BD2D=dats[0].Logs.bds["BD2D"]} mV'))
fig.update_layout(xaxis_title='Heater bias /nA', yaxis_title='Theta /mV',
                      title=f'Dats{min(datnums)}-{max(datnums)}: DC bias')
fig.show(renderer='browser')